In [1]:
!pip install Office365-REST-Python-Client

In [1]:
from office365.runtime.auth.client_credential import ClientCredential
from office365.sharepoint.client_context import ClientContext
from office365.sharepoint.files.file import File
from collections.abc import Iterable

def flatten_list(to_flatten_list):
    for el in to_flatten_list:
        if isinstance(el, Iterable) and not isinstance(el, (str, bytes)):
            yield from flatten_list(el)
        else:
            yield el

def connect_with_website(client_id: str, client_secret: str, site_url: str) -> ClientContext:
    creds = ClientCredential(client_id, client_secret)
    ctx = ClientContext(site_url).with_credentials(creds)
    web = ctx.web
    ctx.load(web)
    ctx.execute_query()
    print('Connected to SharePoint: ',web.properties['Title'])
    return ctx

def get_list_of_folder_paths(ctx, doc_url):
    folders_list = []
    root_folder = ctx.web.get_folder_by_server_relative_url(doc_url)
    folders = root_folder.folders
    ctx.load(folders)
    ctx.execute_query()
    for folder in folders:
        full_path = doc_url + f"/{folder}"
        folders_list.append(folder.properties['ServerRelativeUrl'])
        folders_list.extend(get_list_of_folder_paths(ctx, full_path))
    
    return folders_list

def get_file_path(ctx, folder_url):
    files_list = {}
    root_folder = ctx.web.get_folder_by_server_relative_url(folder_url)
    root_folder.expand(["Files"]).get().execute_query()
    for file in root_folder.files:
        if file.properties['Name'][-4:] != 'aspx':
            files_list[file.properties['Name']] = file.properties['ServerRelativeUrl']
    return files_list

def get_list_of_files_paths(ctx, folders):
    files_list = []
    files_names = []
    for folder in folders:
        files = get_file_path(ctx, folder)
        if files.values():
            files_list.append(files.values())
            files_names.append(files.keys())
    files_list = list(flatten_list(files_list))
    files_names = list(flatten_list(files_names))
    return files_list, files_names

def save_files(ctx, file_names, file_paths, download_path):
    for i, file_p in enumerate(file_paths):
        response = File.open_binary(ctx, file_p)
        save_path = download_path + "\\" + file_names[i]
        with open(save_path, 'wb') as output_file:
            output_file.write(response.content)
    print("Files downloaded")
            

if __name__ == "__main__":
    client_id = ""
    client_secret = ""
    site_url = ""
    folder_url_shrpt = ""
    download_path = r""

    ctx = connect_with_website(client_id, client_secret, site_url)
    folder_paths_list = get_list_of_folder_paths(ctx, folder_url_shrpt)
    file_paths_list, files_names = get_list_of_files_paths(ctx, folder_paths_list)
    save_files(ctx, files_names, file_paths_list, download_path)

Connected to SharePoint:  dev-kno-test
Files downloaded
